# COGS 108 - Data Checkpoint

# Names

- Tyler Le
- Aditya Tomar
- William Lynch
- Michael Mao
- Natalie Quach

<a id='research_question'></a>
# Research Question

Is there a positive correlation between the cost of living and the impact of natural disasters in terms of injuries, casualties, and property damage per capita at the county level? Furthermore, in which state does the impact of natural disasters affect cost of living the most?

# Dataset(s)
Please note that all data file names correspond to their name in our "datasets" folder.

**1. Frequency of Disasters By State**
- Dataset Name: Billion-Dollar Disasters By Year (CPI-Adjusted) (natural_disaster_frequencies.csv)
- Link to the dataset: https://www.ncdc.noaa.gov/billions/state-freq-data.csv
- Number of observations: 2228 
- Number of features: 9. These features are mostly numerical.

This dataset contains the count of natural disasters that cost more than 1 billion dollars for each state from 1980 - 2021. Each observation contains one state and the count of each natural disaster type for a given year.

**2. Types of Disaster By State/County**
- Dataset Name: Disaster Declarations Summaries (DisasterDeclarationsSummaries.csv)
- Link to the dataset: https://www.fema.gov/api/open/v1/DisasterDeclarationsSummaries.csv
- Number of observations: 62771
- Number of features: 23. Contains a combination of numerical and string data types.

This dataset contains all federally declared natural disasters from 1953-2022 (by year, state, county, and type) along with declared recovery programs. Each observation contains the type of natural disaster, when it occurred, and the state/county it occured in.


**3. National Risk Index (NRI)**
- Dataset Name: National Risk Index per County (NRI_Table_Counties.csv)
- Link to the dataset: https://hazards.fema.gov/nri/data-resources#csvDownload
- Number of observations: 3142
- Number of features: 365. Contains a combination of numerical and string data types.

Dataset from FEMA that identifies counties and states most at risk to 18 natural hazards. Includes data about expected annual losses from natural hazards, social vulnerability and community resilience. 

**4. States with Coastline**
- Dataset Name: States with Coastline (states_with_coastline.csv)
- Link to the dataset: https://worldpopulationreview.com/state-rankings/coastal-states
- Number of observations: 50
- Number of features: 2. Contains string data types.

This dataset contains whether or not each state in the United States has a coast. Each observation contains a state and its associated coast.


**5. Cost of Living**
- Dataset Name: Cost of Living (cost_of_living.csv)
- Link to the dataset: https://advisorsmith.com/wp-content/uploads/2021/02/advisorsmith_cost_of_living_index.csv
- Number of observations: 510
- Number of features: 3. Contains numerical and string data types.

Each observation in this dataset contains a state, the city associated with the state, and the Cost of Living Index. The Cost of Living Index measures the costs such as food and energy.

**6. Climate**
- Dataset Name: Average Climate by County (Average_Climate_By_County.csv)
- Link to the dataset: https://www.ncdc.noaa.gov/cag/county/mapping
- Number of observations: 3137
- Number of features: 3. Contains a combination of numerical and string types.

This dataset contains the mean climate (measured in Fahrenheit) over a 5-year span from 2017 to 2022 for all counties in the USA except those in Hawaii. This dataset is for comparing the correlation between natural disasters vs cost of living with climate vs cost of living because climate is a potential confounding variable that affects cost of living.

**6. Housing Price**
- Dataset Name: 
- Link to the dataset: 
- Number of observations: 
- Number of features: 

This dataset contains ...



# Setup

In [2]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm # progress bar for .apply()
import warnings
warnings.filterwarnings('ignore')


# Data Cleaning

**Table #1 (Frequency of Disaster By State):** This dataset was fairly clean in that there were no missing values and each observation contains whether or not a state had a natural disaster event in a certain year. We decided to remove spaces from the column names and to replace them with underscores. Since each column describing a natural disaster contains the count of how many natural disasters of that type, there was little data cleaning needed for this dataset.

**Table #2 (Types of Disasters By State/County):** This dataset was fairly clean. We first focused on extracting the relevant columns, which were "state", "declarationType", "incidentType", "declarationTitle", and "declarationArea". These were the relevant variables since we want each state, the type of natural disaster, and whether it occured on a county level or not. We filtered the dataset to only contain natural disasters that occurred at the county level and standardized the column. We decided to keep the year it happened rather than the exact month and day since in our EDA in the future we would like to explore the natural disaster frequencies by decade. To make future analyses more convienient, we renamed some of the column names. Also, we checked for missing values and found that there were none. 

**Table #3 (National Risk Index):** This dataset was fairly clean. We focused on extracting the relevant columns, such as county, population size, National Risk Index score, and expected annual loss. These variables are important because we would like to compare counties per capita. We also decided to lowercase all the columns and replace spaces with underscores for consistency across all dataframes.

**Table #4 (States with Coastline):** This dataset was fairly clean. Originally, each observation in this dataset contained a state and its associated coast. If the state did not have a coast associated, it had a value "None". To aid with future analyses, such as fitting multiple linear regression models later on, we changed the "coast" column to be binary where 0 means that a state does not have a coast associated with it and 1 means that a state does have a coast associated with it.

**Table #5 (Cost of Living):** This dataset was fairly clean. However, we wanted to look at cost of living as it relates to injuries, casualties, and private property damage broken up by county. Currently, the data we have only shows the cities. We used a geocoder to retrieve the county information. We then made a new column in the dataframe to store counties for each city and rearranged the columns to a favored format.

**Table #6 (Average Climate by County)**
This dataset was very clean. All that we needed to change was remove ID numbers after the state abbreviations and change the column names from "Location ID" to "State", "Location" to "County", and "Value" to "Temperature (F)". 

We plan to merge each of the datasets by either the county or state columns. Additionally, we have datasets containing the number of casualties, number of injuries, and property damage amount per state for the years 2015-2020 that we plan on processing and cleaning after the checkpoint (these datasets are in the "datasets/nat_disast_bystate_deaths_cost” folder).

**Table #7 (Housing Price)**
FILL OUT LATER

## Clean Table #1 (Frequency of Disaster By State)

In [3]:
freq_df = pd.read_csv('datasets/natural_disaster_frequencies.csv')

# replace space with underscores in column names
freq_df.columns = freq_df.columns.str.replace(' ', '_')

# check for NaNs
assert(freq_df.isna().sum().sum() == 0)

freq_df

,year,state,drought,flooding,freeze,severe_storm,tropical_cyclone,wildfire,winter_storm
0,1980,AK,0,0,0,0,0,0,0
1,1980,AL,1,0,0,0,0,0,0
2,1980,AR,1,1,0,0,0,0,0
3,1980,AZ,0,0,0,0,0,0,0
4,1980,CA,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
2221,2021,VT,0,0,0,0,0,0,0
2222,2021,WA,1,0,0,0,0,1,1
2223,2021,WI,0,0,0,4,0,0,0
2224,2021,WV,0,0,0,0,1,0,0


## Clean Table #2 (Types of Disaster By State/County)

In [4]:
def standardize_county(str_in):
    try:
        if '(County)' in str_in:
            output = str_in.replace('(County)','')
        else:
            output = None
    except: 
        output = None

    return output


def standardize_year(str_in):
    try:
        output = str_in.split('T')[0]
        output = pd.to_datetime(str_in).year
    except:
        output = None
        
    return output

In [5]:
disaster_type_df = pd.read_csv('datasets/DisasterDeclarationsSummaries.csv')

# select a subset of the columns
wanted_columns = ['state', 'declarationDate','incidentType','declarationTitle','designatedArea']

# rename the columns
disaster_type_df = disaster_type_df[wanted_columns].rename(columns={"declarationDate":"year", "designatedArea": "county", "incidentType":"disaster_type", "declarationTitle":"disaster_declaration"})

# Set "Statewide" to None and strip "(County)" from all counties
disaster_type_df['county'] = disaster_type_df['county'].apply(standardize_county)

# filter dataset to only include non-null 
disaster_type_df = disaster_type_df[~disaster_type_df['county'].isnull()]

# strip year column to only include year
disaster_type_df['year'] = disaster_type_df['year'].apply(standardize_year)

# sort by year
disaster_type_df = disaster_type_df.sort_values('year').reset_index(drop = True)

# check for no NaNs
assert(disaster_type_df.isna().sum().sum() == 0)

disaster_type_df

,state,year,disaster_type,disaster_declaration,county
0,IN,1959,Flood,FLOOD,Clay
1,WA,1964,Flood,HEAVY RAINS & FLOODING,Wahkiakum
2,WA,1964,Flood,HEAVY RAINS & FLOODING,Skamania
3,WA,1964,Flood,HEAVY RAINS & FLOODING,Pierce
4,WA,1964,Flood,HEAVY RAINS & FLOODING,Pacific
...,...,...,...,...,...
55040,WA,2022,Flood,"SEVERE STORMS, STRAIGHT-LINE WINDS, FLOODING, ...",Whatcom
55041,WA,2022,Flood,"SEVERE STORMS, STRAIGHT-LINE WINDS, FLOODING, ...",Clallam
55042,WA,2022,Flood,"SEVERE STORMS, STRAIGHT-LINE WINDS, FLOODING, ...",Skagit
55043,TN,2022,Tornado,"SEVERE STORMS, STRAIGHT-LINE WINDS, AND TORNADOES",Gibson


## Clean Table #3 (NRI)

In [7]:
# Look at the "NRI Data Dictionary in the datasets/NRI_Table_Counties" to see what the cols mean
# EAL = "Expected Annual Lost", quantifies the anticipated economic damage resulting from natural hazards each year. 
# 1-100 scale

df_nri = pd.read_csv('datasets/NRI_Table_Counties/NRI_Table_Counties.csv')

# select a subset of the columns
wanted_cols = ['STATE','STATEABBRV','STATEFIPS','COUNTY','COUNTYFIPS','POPULATION','AREA','RISK_SCORE','RISK_RATNG','EAL_SCORE','EAL_RATNG']
df_nri = df_nri[wanted_cols]

# lowercase all columns
df_nri.columns = df_nri.columns.str.lower()

# rename columns
df_nri = df_nri.rename(columns={"stateabbrv":"state_abbrv", "risk_ratng":"risk_rating" ,"eal_ratng":"eal_rating","statefips":"state_fips","countyfips":"county_fips"})

# we need state and county FIPS information (already included in NRI dataset)
df_nri['state_fips'] = df_nri['state_fips'].apply(lambda x: str(x).zfill(2))
df_nri['county_fips'] = df_nri['county_fips'].apply(lambda x: str(x).zfill(3))
df_nri['fips'] = df_nri['state_fips'] + df_nri['county_fips']

In [8]:
# Need to convert risk/eal rating to values.
df_nri.value_counts('risk_rating')
encoding_dict = {"Very Low": 1, "Relatively Low":2, "Relatively Moderate": 3, "Relatively High": 4, "Very High": 5}

def encode_ratings(str_in):
    return (encoding_dict[str_in])

In [57]:
df_nri['risk_encoded'] = df_nri['risk_rating'].apply(encode_ratings)
df_nri['eal_encoded'] = df_nri['eal_rating'].apply(encode_ratings)

# make sure there are no NaNs
assert(df_nri.isna().sum().sum() == 0)

In [58]:
df_nri.to_csv("datasets/cleaned/nri.csv",index=False)

In [59]:
df_nri.head()

,state,state_abbrv,state_fips,county,county_fips,population,area,risk_score,risk_rating,eal_score,eal_rating,fips,risk_encoded,eal_encoded
0,Kentucky,KY,21,Johnson,115,23356,261.958144,9.281419,Relatively Low,11.921944,Relatively Low,21115,2,2
1,Kentucky,KY,21,Kenton,117,159720,160.213975,10.449057,Relatively Low,16.837131,Relatively Moderate,21117,2,3
2,Kentucky,KY,21,Knott,119,16346,351.517978,10.068395,Relatively Low,10.945913,Relatively Low,21119,2,2
3,Kentucky,KY,21,Knox,121,31883,386.298435,11.858245,Relatively Low,11.983719,Relatively Low,21121,2,2
4,Kentucky,KY,21,Larue,123,14193,261.539564,4.610900,Very Low,7.028611,Very Low,21123,1,1


## Clean Table #4 (States with Coastline)

In [60]:
coastline_df = pd.read_csv('datasets/states_with_coastline.csv')
coastline_df

,State,coast
0,Alabama,Gulf Coast
1,Alaska,Pacific Ocean
2,Arizona,None
3,Arkansas,None
4,California,Pacific Ocean
5,Colorado,None
6,Connecticut,Atlantic Ocean
7,Delaware,Atlantic Ocean
8,Florida,Atlantic Ocean/Gulf Coast
9,Georgia,Atlantic Ocean


In [61]:
# check data types for each column
coastline_df.dtypes

State    object
coast    object
dtype: object

In [62]:
coastline_df['coast'].value_counts()

None                                20
Atlantic Ocean                      12
Great Lakes Coast                    7
Pacific Ocean                        5
Gulf Coast                           4
Atlantic Ocean/Gulf Coast            1
Atlantic Ocean/Great Lakes Coast     1
Name: coast, dtype: int64

In [63]:
# check for missing values
coastline_df.isna().sum()

State    0
coast    0
dtype: int64

In [64]:
# Binarize "coast" column. 0 = no coastline, 1 = yes coastline
def clean_coast(coast_val):
    coast_val = coast_val.lower()

    if "none" in coast_val:
        coast_val = coast_val.replace("none", "0")
        output = int(coast_val)
    else:
        output = 1
    
    return output

In [65]:
# test function from above
assert clean_coast('None') == 0
assert clean_coast('Atlantic Ocean') == 1

In [66]:
coastline_df['coast'] = coastline_df['coast'].apply(clean_coast)

In [67]:
coastline_df

,State,coast
0,Alabama,1
1,Alaska,1
2,Arizona,0
3,Arkansas,0
4,California,1
5,Colorado,0
6,Connecticut,1
7,Delaware,1
8,Florida,1
9,Georgia,1


## Clean Table #5 (Cost of Living)

In [68]:
df = pd.read_csv('datasets/cost_of_living.csv')
df["city_state"] = df["City"] + ", " + df["State"]
df

,City,State,Cost of Living Index,city_state
0,Abilene,TX,89.1,"Abilene, TX"
1,Adrian,MI,90.5,"Adrian, MI"
2,Akron,OH,89.4,"Akron, OH"
3,Alamogordo,NM,85.8,"Alamogordo, NM"
4,Albany,GA,87.3,"Albany, GA"
...,...,...,...,...
505,Wheeling,WV,84.1,"Wheeling, WV"
506,New London,CT,105.9,"New London, CT"
507,Daphne,AL,96.6,"Daphne, AL"
508,Victoria,TX,89.5,"Victoria, TX"


In [69]:
geolocator = Nominatim(user_agent='find-county')

# extract County from Geocode object 
def standardize_geocode(location):
    try:
        output = [x.strip() for x in location.address.split(',') if 'County' in x ][0]
    except: 
        output = None
        
    return output

In [70]:
# we need this to avoid timeout 
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# allow for progress bar on .apply()
tqdm.pandas()

# convert city_state column to geocode
try:
    df['geocode'] = df['city_state'].progress_apply(geocode)
except:
    pass

 75%|███████▍  | 381/510 [06:21<02:39,  1.23s/it]RateLimiter caught an error, retrying (0/2 tries). Called with (*('San Jose, CA',), **{}).
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "/opt/anaconda3/lib/python3.8/http/client.py", line 1347, in getresponse
    response.begin()
  File "/opt/anaconda3/lib/python3.8/http/client.py", line 307, in begin
    version, status, reason = self._read_status()
  File "/opt/anaconda3/lib/python3.8/http/client.py", line 268, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/opt/anaconda3/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
  File "/opt/anaconda3/lib/pyth

In [71]:
# convert geocode column to extract the county
df['county'] = df['geocode'].apply(standardize_geocode)

In [72]:
# check for nans
df.isna().sum().sum()

34

In [73]:
# remove rows with nans
df.dropna(how='any')

,City,State,Cost of Living Index,city_state,geocode,county
0,Abilene,TX,89.1,"Abilene, TX","(Abilene, Taylor County, Texas, 79697, United ...",Taylor County
1,Adrian,MI,90.5,"Adrian, MI","(Adrian, Lenawee County, Michigan, 49221, Unit...",Lenawee County
2,Akron,OH,89.4,"Akron, OH","(Akron, Summit County, Ohio, United States, (4...",Summit County
3,Alamogordo,NM,85.8,"Alamogordo, NM","(Alamogordo, Otero County, New Mexico, 88310, ...",Otero County
4,Albany,GA,87.3,"Albany, GA","(Albany, Dougherty County, Georgia, 31701, Uni...",Dougherty County
...,...,...,...,...,...,...
505,Wheeling,WV,84.1,"Wheeling, WV","(Wheeling, Ohio County, West Virginia, 26003, ...",Ohio County
506,New London,CT,105.9,"New London, CT","(New London, New London County, Connecticut, 0...",New London County
507,Daphne,AL,96.6,"Daphne, AL","(Daphne, Baldwin County, Alabama, 35626, Unite...",Baldwin County
508,Victoria,TX,89.5,"Victoria, TX","(Victoria County, Texas, United States, (28.80...",Victoria County


In [76]:
# # save resulting df as csv to avoid running previous cells frequently
# df.to_csv('datasets/col_updated.csv', index=False)
df.to_csv('datasets/cleaned/col_updated.csv', index=False)
df

,City,State,Cost of Living Index,city_state,geocode,county
0,Abilene,TX,89.1,"Abilene, TX","(Abilene, Taylor County, Texas, 79697, United ...",Taylor County
1,Adrian,MI,90.5,"Adrian, MI","(Adrian, Lenawee County, Michigan, 49221, Unit...",Lenawee County
2,Akron,OH,89.4,"Akron, OH","(Akron, Summit County, Ohio, United States, (4...",Summit County
3,Alamogordo,NM,85.8,"Alamogordo, NM","(Alamogordo, Otero County, New Mexico, 88310, ...",Otero County
4,Albany,GA,87.3,"Albany, GA","(Albany, Dougherty County, Georgia, 31701, Uni...",Dougherty County
...,...,...,...,...,...,...
505,Wheeling,WV,84.1,"Wheeling, WV","(Wheeling, Ohio County, West Virginia, 26003, ...",Ohio County
506,New London,CT,105.9,"New London, CT","(New London, New London County, Connecticut, 0...",New London County
507,Daphne,AL,96.6,"Daphne, AL","(Daphne, Baldwin County, Alabama, 35626, Unite...",Baldwin County
508,Victoria,TX,89.5,"Victoria, TX","(Victoria County, Texas, United States, (28.80...",Victoria County


## Clean Table #6

In [77]:
# Dataset for the mean climate for each county in the USA except those in Hawaii. The climate is measured over a 5-year span from 2017-2022. 
climate_df = pd.read_csv('datasets/Average_Climate_By_County.csv')
climate_df['Location ID'] = climate_df['Location ID'].apply([lambda x: x[0:2]])
climate_df = climate_df.rename(columns={'Location ID': 'State', 'Location': 'County', 'Value': 'Temperature (F)'})
climate_df

,State,County,Temperature (F)
0,AL,Autauga County,65.7
1,AL,Baldwin County,68.4
2,AL,Barbour County,66.0
3,AL,Bibb County,63.9
4,AL,Blount County,62.7
...,...,...,...
3132,AK,Skagway Municipality,30.9
3133,AK,Southeast Fairbanks Census Area,25.7
3134,AK,Wrangell City and Borough,40.0
3135,AK,Yakutat City and Borough,33.9
